In [23]:
# Imports
import os
import numpy as np
import pandas as pd
import keras
from keras import layers, models

# Step 1: Data Processing

In [24]:
image_shape = (500, 500, 3)  # Create tuple for image size

# Establish paths
data_folder = os.path.join("..", "data")
test_path = os.path.join(data_folder, "test")
train_path = os.path.join(data_folder, "train")
valid_path = os.path.join(data_folder, "valid")

# Load the images sets
test_images = keras.utils.image_dataset_from_directory(
    directory=test_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)
train_images = keras.utils.image_dataset_from_directory(
    directory=train_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)
valid_images = keras.utils.image_dataset_from_directory(
    directory=valid_path,
    labels="inferred",
    image_size=(image_shape[0], image_shape[1]),
)

Found 539 files belonging to 3 classes.
Found 1942 files belonging to 3 classes.
Found 431 files belonging to 3 classes.


# Step 2: Neural Network Architecture Design

In [25]:
# Initialize the model
model = models.Sequential()

# Add an input layer
model.add(layers.Input(shape=image_shape))

# Add convolutional layer with 32 filters, 3x3 kernel, ReLU activation
model.add(layers.Conv2D(32, (3, 3), activation='relu'))

# Add max pooling layer with 2x2 pool size
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(layers.Flatten())

# Add fully connected layer with 128 units and ReLU activation
model.add(layers.Dense(128, activation='relu'))

# Add output layer with 3 units (number of classes) and softmax activation
model.add(layers.Dense(3, activation='softmax'))

# Step 3: Hyperparameter Analysis

In [26]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Display the model's architecture
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 498, 498, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 249, 249, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1984032)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │   253,956,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 253,957,507 (968.77 MB)

 Trainable params: 253,957,507 (968.77 MB)

 Non-trainable params: 0 (0.00 B)

# Model Evaluation

In [22]:
# Train the model
history = model.fit(
    x=train_images,
    epochs=5,
    batch_size=128,
    validation_data=valid_images,
)

Epoch 1/5


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 3)